Updates:
- 24 Oct: Start
    - Shows Envelope Number is the most granular order level
    - Shows more than 90 % of Shipments conducted during 6am - 6pm
    - Shows More than 90% of Envelopes being shipped out on single Loadout Time
    - filter out envelope with total pallets less than 12, which occupy less than half of  the row
    - Derived LO Rows needed by each envelope 
    

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from scipy.stats import skew,kurtosis
import datetime
import re
import fnmatch
import math

In [ ]:
%matplotlib inline
# to view all columns
pd.set_option('display.max_columns',500)
plt.style.use('seaborn')

In [ ]:
def summary_object(arg_df):
    
    object_list=[]
    category_list=[]
    bool_list=[]
    unilabel_list=[]
    missing_list=[]
    
    for c in arg_df.columns:
        if arg_df[c].dtypes==object:
            object_list.append(c)
        elif str(arg_df[c].dtypes)=='category':
            category_list.append(c)
        elif arg_df[c].dtypes==bool:
            bool_list.append(c)
    if len(object_list)+len(category_list)+len(bool_list)>0:    
        index_list=['Count','Unique','Missing (%)','Top','Top (%)','Bottom','Bottom (%)']
        df_summary=pd.DataFrame(data=np.zeros((len(index_list),len(object_list))),index=index_list,columns=object_list)

        for col in object_list+category_list+bool_list:
            vc=arg_df[col].value_counts().reset_index()
            df_summary.loc['Count',col]=(arg_df[col].count())
            df_summary.loc['Unique',col]=len(arg_df[col].unique())
            df_summary.loc['Missing (%)',col]=arg_df[col].isna().mean()*100
            df_summary.loc['Top',col]=vc.iloc[0,0]
            df_summary.loc['Top (%)',col]=vc.iloc[0,1]/len(arg_df)*100
            if len(arg_df[col].unique())>2:
                df_summary.loc['Bottom',col]=vc.iloc[-1,0]
                df_summary.loc['Bottom (%)',col]=vc.iloc[-1,1]/len(arg_df)*100
            elif (len(arg_df[col].unique())==2) & (df_summary.loc['Missing (%)',col]==0):
                df_summary.loc['Bottom',col]=vc.iloc[-1,0]
                df_summary.loc['Bottom (%)',col]=vc.iloc[-1,1]/len(arg_df)*100
            else:
                unilabel_list.append(col)
            if df_summary.loc['Missing (%)',col]==100:
                missing_list.append(col)
                
        df_summary=df_summary.T.sort_values(['Missing (%)','Unique'],ascending=False)
        df_summary=df_summary[(df_summary['Unique']>1) & (df_summary['Missing (%)']!=100)]
        df_summary.reset_index(inplace=True)
        df_summary.index=df_summary.index+1
        df_summary.columns=['Attribute']+index_list
        
        print('SUMMARY OF {} NON-NUMERICAL ATTRIBUTES:\n'.format(
            len(object_list)+len(category_list)+len(bool_list)))
        if len(object_list)>0:
            print('{} Object Columns'.format(len(object_list)))
        if len(category_list)>0:
            print('{} Categorical Columns'.format(len(category_list)))
        if len(bool_list)>0:
            print('{} Bool Columns'.format(len(bool_list)))
        if len(unilabel_list)>0:
            print('\n{} Columns with Single Label : \n{}'.format(len(unilabel_list),unilabel_list))
        if len(missing_list)>0:
            print('\n{} Empty Columns: \n{}'.format(len(missing_list),missing_list))   
            
        del arg_df,object_list,vc,index_list,unilabel_list,missing_list
        gc.collect()
        return df_summary
    else:
        print('No Non-Numerical Attributes')
'============================================================='        
def summary_numerical(arg_df):

    target_list=[]
    missing_list=[]
    zero_skew_list=[]
    
    for c in arg_df.columns:
        datatype=arg_df[c].dtypes
        if datatype != object and datatype != bool and str(datatype) != 'category' and str(datatype) !='datetime64[ns]':
            target_list.append(c)
    if len(target_list)>0:
        from scipy.stats import skew,kurtosis
        
        index_list=['Count','Missing (%)','Mean','Median','Min','Max','Skewness','Kurtosis']
        df_summary=pd.DataFrame(data=np.zeros((len(index_list),len(target_list))),
                                index=index_list,columns=target_list)
        for col in target_list:
            df_summary.loc['Count',col]=arg_df[col].count()
            df_summary.loc['Missing (%)',col]=arg_df[col].isna().mean()*100
            if df_summary.loc['Missing (%)',col]!=100:
                df_summary.loc['Mean',col]=arg_df[col].mean()
                df_summary.loc['Median',col]=arg_df[col].median()
                df_summary.loc['Min',col]=arg_df[col].min()
                df_summary.loc['Max',col]=arg_df[col].max()
                df_summary.loc['Skewness',col]=skew(arg_df[col])
                if df_summary.loc['Skewness',col]==0:
                    zero_skew_list.append(col)
                df_summary.loc['Kurtosis',col]=kurtosis(arg_df[col])
            else:
                missing_list.append(col)
                
        df_summary=df_summary.T.sort_values(['Missing (%)','Skewness'],ascending=False)
        df_summary=df_summary[(df_summary['Skewness']!=0) & (df_summary['Missing (%)']!=100)]
        df_summary.reset_index(inplace=True)
        df_summary.index=df_summary.index+1
        df_summary.columns=['Attribute']+index_list
        
        print('SUMMARY OF {} NUMERICAL ATTRIBUTES:'.format(len(target_list)))
        if len(zero_skew_list)>0:
            print('\n{} Columns with Single Value: \n{}'.format(len(zero_skew_list),zero_skew_list))
        if len(missing_list)>0:
            print('\n{} Empty Columns: \n{}'.format(len(missing_list),missing_list))
        del arg_df,target_list,index_list
        gc.collect()

        return df_summary
    else:
        print('No Numerical Attributes')
'==================================================================='        
def drop_unilable_column(arg_df):
    
    target_list=[]
    object_list=[]
    number_list=[]
    for c in arg_df.columns:
        if (arg_df[c].dtypes==object) | (str(arg_df[c].dtypes)=='category') | (arg_df[c].dtypes==bool):
            object_list.append(c)
        elif str(arg_df[c].dtypes)!='datetime64[ns]':
            number_list.append(c)
    if len(object_list)>0:    
        for c in object_list:
            if len(arg_df[c].unique())==1:
                target_list.append(c)
            elif (len(arg_df[c].unique())==2) & (arg_df[c].isna().mean()>0):
                target_list.append(c)
    
    if len(number_list)>0:   
        from scipy.stats import skew
        for c in number_list:
            if skew(arg_df[c])==0:
                target_list.append(c)
                
    if len(target_list)>0:
        arg_df.drop(columns=target_list,axis='columns',inplace=True)
        print('Drop {} Columns with Single Label:\n{}'.format(len(target_list),target_list))
    else: 
        print('No Columns with Single Label/Value')

    del target_list,object_list
'===================================================================' 
def drop_empty_column(arg_df):
    target_list=[]
    for c in arg_df.columns:
        if arg_df[c].count()==0:
            target_list.append(c)
    if len(target_list)>0:
        arg_df.drop(columns=target_list,axis=1,inplace=True)
        print('Delete {} Empty Column : \n{}'.format(len(target_list),target_list))
    else:
        print('No Empty Column')
'==================================================================='        
def drop_columns(arg_df,column_names):
    arg_df.drop(columns=column_names,axis='columns',inplace=True)
    print('Drop {} columns : \n{}'.format(len(column_names),column_names))
'============================================================='        
def extract_room_row(arg_df,col_position):
    '''To return unique Room-Row from standard Room-Row-Column-Height position data'''
    roomrow=[]
    roomrow=arg_df[col_position].apply(lambda x :x.split('-')[0]+'-'+x.split('-')[1] if '-' in x else x)
    roomrow=roomrow.unique().tolist()
    roomrow=pd.DataFrame(roomrow,columns=['Unique_Row']).sort_values(by='Unique_Row')
    return roomrow['Unique_Row'].values
'============================================================='  
def convert_room_row(position):
    if '-' in position:
        w,x,y,z=position.split('-')
        return w+'-'+x
    else:
        return position
'============================================================='  
def generate_dif_columns(arg_df,left_column,left_sffx,right_sffx):
    common_title=[]
    for idx,c in enumerate(left_column):
        common_title.append(left_column[idx].split(left_sffx)[0])
    print('There are {} common columns : \n{}'.format(len(left_column),common_title))

    for idx,c in enumerate(common_title):
        compare_col=common_title[idx].replace(' ','')
        arg_df['dif_'+compare_col]=(arg_df[c+left_sffx]!=arg_df[c+right_sffx]) & (arg_df[c+right_sffx].notna())
    print('\nColumns Generated : {}'.format(len(common_title)))
'==================================================================='        
def find_time_dif_hour(arg_df,ref_date,proc_date):
    new_date_attribute=[c.replace(' ','_') for c in proc_date]
    for idx,c in enumerate(proc_date):
        arg_df[new_date_attribute[idx]+'_hour']=arg_df[c]-arg_df[ref_date]
        arg_df[new_date_attribute[idx]+'_hour']=arg_df[new_date_attribute[idx]+'_hour'].astype('timedelta64[h]')

# 1) Import 2 dataset : trans and disp

In [ ]:
path='C:/Users/Nan/Documents/GitHub_Data/'
file_1=path+'p_transaction_2.csv'
file_2=path+'p_dispatched_2.csv'

filename_1=file_1
filename_2=file_2

sffx_transaction='_Trsc'
sffx_dispatched='_Dptch'
target_process='FW'

In [ ]:
filename_1

In [ ]:
trans=pd.read_csv(filename_1)
disp=pd.read_csv(filename_2)

In [ ]:
trans.shape

In [ ]:
trans.head(2)

In [ ]:
disp.shape

In [ ]:
disp.head(2)

### Data Processing so that common columns have same set of data label
    -change to string:Fruit Size Code_Trsc
    -truncate 0 at first position:Purchase Pool Code_Trsc
    -incorrect calculation as referring to dif,should remain in datetime: date columns(incl Ok_Until_Date_day,Pack_Date_day)

In [ ]:
trans['Fruit Size Code']=trans['Fruit Size Code'].astype('str')
disp['Fruit Size Code']=disp['Fruit Size Code'].astype('str')

In [ ]:
trans['Purchase Pool Code']=trans['Purchase Pool Code'].apply(lambda x:x.split('0')[1] if x[0]=='0' else x)
disp['Purchase Pool Code']=disp['Purchase Pool Code'].apply(lambda x:x.split('0')[1] if x[0]=='0' else x)

In [ ]:
#drop_columns(disp,['Pack_Date_day', 'Ok_Until_Date_day'])
drop_columns(disp,['Pack_Date_day'])

In [ ]:
# to prevent OK Until Date with same date but dif time
trans['Ok Until Date']=pd.to_datetime(trans['Ok Until Date'],format='%Y-%m-%d %H:%M:%S')
disp['Ok Until Date']=pd.to_datetime(disp['Ok Until Date'],format='%Y-%m-%d %H:%M:%S')

In [ ]:
trans['Ok Until Date']=trans['Ok Until Date'].dt.date
disp['Ok Until Date']=disp['Ok Until Date'].dt.date

In [ ]:
trans['Transaction Date Time']=pd.to_datetime(trans['Transaction Date Time'],format='%Y-%m-%d %H:%M:%S')

In [ ]:
disp.head()

In [ ]:
disp['Location Height'].value_counts()

In [ ]:
disp['Location Column']=disp['Location Column'].map(lambda x:x[:-2] if x!='na' else x)
disp['Location Height']=disp['Location Height'].map(lambda x:x[:-2] if x!='na' else x)

In [ ]:
df_disp=disp[['Loadout Date','Load Start Date','Pallet Number','Order Number','Envelope Number','Container','Container Number',
                'Location Room Code','Location Row Code','Location Column',
                'Location Height','Loadout Priority','Shipment Type Code','Destination Port Code',
               'Trucking Company Code','Stacking Configuration Code','Pack Type Code']]

In [ ]:
df_disp.head()

### Derive Last Value

In [ ]:
df_disp.loc[df_disp['Location Column']!='na']['Location Room Code'].value_counts()

In [ ]:
criterion=df_disp['Location Room Code'].map(lambda x:x.startswith('Q'))

In [ ]:
df_disp.loc[criterion,'Last Value']=df_disp['Location Room Code']+'-'+df_disp['Location Row Code']+'-'+df_disp['Location Column']+'-'+df_disp['Location Height']


In [ ]:
df_disp.loc[df_disp['Last Value'].isna(),'Last Value']=df_disp['Location Room Code']

In [ ]:
df_disp.head()

In [ ]:
df_disp['RoomRow_Last Value']=df_disp['Last Value'].map(convert_room_row)

### To investigate the distribution of loadoat time
- Findings : 96.74 % of Shipments conducted during 6am - 6pm

In [ ]:
df_disp['Loadout Date']=pd.to_datetime(df_disp['Loadout Date'],format='%Y-%m-%d %H:%M:%S')

In [ ]:
df_disp['Load Hour']=df_disp['Loadout Date'].dt.hour

In [ ]:
df_disp['Load Hour'].plot(kind='hist',bins=50)

In [ ]:
print('Percentage of Shipments conducted during day shift is : {:.2f} %'.
      format(len(df_disp[(df_disp['Load Hour']>=6) & (df_disp['Load Hour']<=18)])/len(df_disp)*100))

### Investigate Relationship of  _Order Number_,_Container Number_ and _Envelope Number_ 
- Findings : Envelope Number indicate the most granular level of order structure
- There are two types of order, container order and chartered order
- Container order contains envelope number and container number
- Chartered order contains envelope number only


In [ ]:
temp_df=df_disp.groupby(['Order Number','Container Number','Envelope Number'])['Pallet Number'].count()
temp_df=temp_df.to_frame()

In [ ]:
temp_df[88:100]

### Investigate Total Number of Shipment Date per Envelope
- Findings : More than 90% of Envelopes being shipped out on single Loadout Time

In [ ]:
temp_df=df_disp.groupby(['Envelope Number','Loadout Date'])['Pallet Number'].count()
temp_df=temp_df.to_frame().reset_index()
total_shipment_per_envelope=temp_df['Envelope Number'].value_counts().to_frame().reset_index()
print('Envelope that shipped on single shipment : {:.2f} %'.format(
len(total_shipment_per_envelope[total_shipment_per_envelope['Envelope Number']==1])/len(total_shipment_per_envelope)*100))

### Investigate the delivery Date of Order 
- Findings : The Loadout Time of envelopes that belongs to same order number could be different although all of the envelopes RTG at same time

In [ ]:
temp_df=df_disp.groupby(['Order Number','Envelope Number','Loadout Date'])['Pallet Number'].count()

In [ ]:
temp_df

In [ ]:
df_disp[df_disp['Envelope Number']==199288]

In [ ]:
df_disp.groupby('Envelope Number')['RoomRow_Last Value'].unique()

In [ ]:
df_disp.loc[df_disp['Envelope Number']==199316,:]

In [ ]:
# hightop pallets also being placed in normal LO Room
df_disp[df_disp['Stacking Configuration Code']!='N']['RoomRow_Last Value'].value_counts()

### Distribution of Total Pallet of all dispatched Envelope
    - More than 50 % of order belongs to Container order of 20 pallets 
    - followed by chartered order of 24 pallets

In [ ]:
df_disp.groupby('Envelope Number')['Pallet Number'].count().plot(kind='hist',bins=50)

In [ ]:
temp=df_disp.groupby('Envelope Number')['Pallet Number'].count().to_frame()

In [ ]:
print('Percentage of Orders with 20 pallet = {:.2f} %'.format(len(temp[temp['Pallet Number']==20])/len(temp)))
print('Percentage of Orders with 24 pallet = {:.2f} %'.format(len(temp[temp['Pallet Number']==24])/len(temp)))

In [ ]:
df_disp.head()

### Filter out Envelope with Total Pallets  of less than 12 pallets, which will occupied at least half of the row when double-stacked
    - data  july 2019 : will filter out 10 % of envelopes

In [ ]:
min_pallet=12

In [ ]:
temp=df_disp.groupby('Envelope Number')['Pallet Number'].count().to_frame().reset_index()

In [ ]:
print('Quantity of Envelope of Original Dataset = {}'.format(temp.shape[0]))
print('Quantity of Envelope of Filtered Dataset = {}'.format(temp[temp['Pallet Number']>=min_pallet].shape[0]))
print('Reduction % = {:.4f} %'.format((temp.shape[0]-temp[temp['Pallet Number']>=min_pallet].shape[0])/temp.shape[0]))

In [ ]:
temp=temp.loc[temp['Pallet Number']>=min_pallet,'Envelope Number']

In [ ]:
temp

In [ ]:
df_disp=df_disp.merge(temp,how='inner',on='Envelope Number')

### Derive isHighTops so that stack type 'L' and 'N' can be grouped together

In [ ]:
df_disp['Stacking Configuration Code'].value_counts()

In [ ]:
df_disp.sample(5)

In [ ]:
df_disp['isHighTops']=df_disp['Stacking Configuration Code'].isin(['1','2','3'])

## Derive LO Row needed for all envelopes
    - Rule : Hightops cannot be double stacked in LO Room
    - Rule : MB packstyle can be double stacked

In [ ]:
# this shows MB pallet can be double-stacked
print(trans[trans['Pack Style Code'].isin(['MB'])]['Previous_Height'].value_counts())

# although the result shows hightops is being double-stacked, it is violating the rules especially in LO ROom
print(trans[~(trans['Stacking Configuration Code'].isin(['N']))]['Previous_Height'].value_counts())

In [ ]:
temp=df_disp.groupby(['Envelope Number','isHighTops'])['Pallet Number'].count().to_frame().reset_index()

In [ ]:
temp

### Derive 'Cells Needed' that handle business rule of all pallets can be double-stacked except HighTops

In [ ]:

temp.loc[temp['isHighTops']==False,'Cells Needed']=temp['Pallet Number']/2
temp.loc[temp['isHighTops']==True,'Cells Needed']=temp['Pallet Number']

In [ ]:
temp['Cells Needed']=temp['Cells Needed'].map(lambda x:math.ceil(x))

In [ ]:
temp

In [ ]:
df_disp_cells=temp.groupby('Envelope Number')['Cells Needed'].sum().to_frame().reset_index()

In [ ]:
df_disp_cells.rename(index=str,columns={'Cells Needed':'Total Cells'},inplace=True)

In [ ]:
df_disp_cells['Total Cells'].plot(kind='Hist',bins=30)

In [ ]:
df_disp_cells['Total Cells'].value_counts()

### Derive 'Row Needed' by dividing 'Total Cells' by 12 and round up

In [ ]:
df_disp_cells['Row Needed']=df_disp_cells['Total Cells']/12

In [ ]:
df_disp_cells.head()

In [ ]:
df_disp_cells['Row Needed']=df_disp_cells['Row Needed'].map(lambda x:math.ceil(x))

In [ ]:
df_disp_cells['Row Needed'].plot(kind='hist')

In [ ]:
df_disp_cells['Row Needed'].describe()

# Merge with Trans

### Merge disp_temp with trans_temp to filter out pallet number in common

In [ ]:
trans_temp=trans[['Transaction Date Time','Pallet Number','New Value','Previous Value']]

In [ ]:
print('Total Transactions Line : {}'.format(trans_temp.shape[0]))
print('Total Number of Pallet : {} '.format(len(trans_temp['Pallet Number'].unique())))

In [ ]:
trans_merge=trans_temp.merge(df_disp['Pallet Number'],how='inner',on='Pallet Number')

In [ ]:
print('Total Transactions Line : {}'.format(trans_merge.shape[0]))
print('Total Number of Pallet : {} '.format(len(trans_merge['Pallet Number'].unique())))

In [ ]:
trans_last=trans_merge.groupby('Pallet Number')[['New Value','Previous Value','Transaction Date Time']].last()

In [ ]:
trans_last.reset_index(inplace=True)

In [ ]:
trans_last.head(3)

### Merge disp_temp with trans_last to populate last position in trans with disp_temp

In [ ]:
disp_merge=df_disp.merge(trans_last,how='left',on='Pallet Number')

In [ ]:
print('Total number of dispatched pallet with no trans record : {}'.format(len(disp_merge[disp_merge['New Value'].isna()])))
print('Total number of dispatched pallet with trans record : {}'.format(len(disp_merge[disp_merge['New Value'].notna()])))

### Filter out data with last position in trans is available

In [ ]:
disp_merge=disp_merge.loc[disp_merge['New Value'].notna()]
disp_merge.shape[0]

In [ ]:
disp_merge.sample(5)

In [ ]:
disp_simple=disp_merge[['Loadout Date','Load Start Date','Pallet Number','Last Value','New Value','Previous Value','Transaction Date Time','Order Number','Envelope Number','Container Number','Stacking Configuration Code','Pack Type Code']]

In [ ]:
disp_simple['RoomRow_New Value']=disp_simple['New Value'].map(convert_room_row)
disp_simple['RoomRow_Previous Value']=disp_simple['Previous Value'].map(convert_room_row)

### Does Pallets that belongs to same Envelope Number being taken from same rows prior to shipment ?
    - part of packhouse SOP
    - hard to extract this part of information from data as the LO Row info could be in any of last few transactions

In [ ]:
temp=disp_simple.groupby(['Envelope Number','RoomRow_Previous Value'])['Pallet Number'].count().to_frame().reset_index()

In [ ]:
temp[100:120]

    - more than 80 % of pallet dispatched have Last Value = New Value
    - hightop pallets also being placed in normal LO Room

In [ ]:
# more than 80 % of transaction have Last Value = New Value
(disp_simple['Last Value']==disp_simple['New Value']).sum()/len(disp_simple)

In [ ]:
disp_simple[disp_simple['Last Value']!=disp_simple['New Value']]